# Chapter 29: Cross-tabulation Deep Dive

In [1]:
import pandas as pd
import numpy as np
import catboost as cb
import collections

url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
   '2020-jetbrains-python-survey.csv'
jb = pd.read_csv(url)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_11284\2686139537.py:8: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [2]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [3]:
def prep_for_ml(df):
    # remove pandas types
    return (df
     .assign(**{col:df[col].astype(float) 
               for col in df.select_dtypes('number')},
             **{col:df[col].astype(str).fillna('') 
               for col in df.select_dtypes(['object', 'category'])})
    )

In [4]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [5]:
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
    )    

In [6]:
jb2 = tweak_jb(jb)

Learning rate set to 0.5
0:	learn: 2.9695218	total: 167ms	remaining: 3.17s
1:	learn: 2.8766539	total: 217ms	remaining: 1.95s
2:	learn: 2.8387189	total: 271ms	remaining: 1.53s
3:	learn: 2.8028751	total: 320ms	remaining: 1.28s
4:	learn: 2.7899957	total: 368ms	remaining: 1.1s
5:	learn: 2.7749439	total: 414ms	remaining: 966ms
6:	learn: 2.7719128	total: 461ms	remaining: 856ms
7:	learn: 2.7649792	total: 509ms	remaining: 763ms
8:	learn: 2.7649588	total: 545ms	remaining: 667ms
9:	learn: 2.7630617	total: 603ms	remaining: 603ms
10:	learn: 2.7625779	total: 648ms	remaining: 531ms
11:	learn: 2.7515902	total: 693ms	remaining: 462ms
12:	learn: 2.7513459	total: 740ms	remaining: 399ms
13:	learn: 2.7445634	total: 784ms	remaining: 336ms
14:	learn: 2.7443257	total: 827ms	remaining: 276ms
15:	learn: 2.7423142	total: 868ms	remaining: 217ms
16:	learn: 2.7419143	total: 912ms	remaining: 161ms
17:	learn: 2.7399387	total: 957ms	remaining: 106ms
18:	learn: 2.7384296	total: 1s	remaining: 52.7ms
19:	learn: 2.738359

## 29.1 Cross-tabulation Summaries

In [8]:
# summariing the count of respondents by country and age
pd.crosstab(index=jb2.country_live, columns=jb2.age)

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,2,7,5,3,0,1
Argentina,1,38,44,20,5,1
Armenia,1,13,3,0,0,0
Australia,4,58,110,63,30,9
Austria,1,31,62,22,12,0
...,...,...,...,...,...,...
United States,40,753,1042,478,264,120
Uruguay,0,6,13,1,0,0
Uzbekistan,0,4,0,0,0,0


## 29.2 Adding Margins

- Both ``.pivot_table`` and ``crosstab`` have margins parameter that will put in a column and row at the right and bottom respectively that summarize the data

In [9]:
pd.crosstab(index=jb2.country_live, columns=jb2.age,
            margins=True)

age,18,21,30,40,50,60,All
country_live,,,,,,,
Algeria,2,7,5,3,0,1,18
Argentina,1,38,44,20,5,1,109
Armenia,1,13,3,0,0,0,17
Australia,4,58,110,63,30,9,274
Austria,1,31,62,22,12,0,128
...,...,...,...,...,...,...,...
Uruguay,0,6,13,1,0,0,20
Uzbekistan,0,4,0,0,0,0,4
Venezuela,1,10,4,5,2,0,22


## 29.3 Normalizing Results

- Calculates the percent of each cell

In [10]:
pd.crosstab(index=jb2.country_live, columns=jb2.age,
            normalize=True)

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,0.000146,0.000511,0.000365,0.000219,0.000000,0.000073
Argentina,0.000073,0.002771,0.003209,0.001459,0.000365,0.000073
Armenia,0.000073,0.000948,0.000219,0.000000,0.000000,0.000000
Australia,0.000292,0.004230,0.008023,0.004595,0.002188,0.000656
Austria,0.000073,0.002261,0.004522,0.001605,0.000875,0.000000
...,...,...,...,...,...,...
United States,0.002917,0.054919,0.075997,0.034863,0.019255,0.008752
Uruguay,0.000000,0.000438,0.000948,0.000073,0.000000,0.000000
Uzbekistan,0.000000,0.000292,0.000000,0.000000,0.000000,0.000000


- We can specify the axis to normalize

In [11]:
pd.crosstab(index=jb2.country_live, columns=jb2.age,
            normalize='columns')

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,0.006349,0.001328,0.000989,0.001479,0.000000,0.004505
Argentina,0.003175,0.007211,0.008706,0.009862,0.006083,0.004505
Armenia,0.003175,0.002467,0.000594,0.000000,0.000000,0.000000
Australia,0.012698,0.011006,0.021765,0.031065,0.036496,0.040541
Austria,0.003175,0.005882,0.012268,0.010848,0.014599,0.000000
...,...,...,...,...,...,...
United States,0.126984,0.142884,0.206173,0.235700,0.321168,0.540541
Uruguay,0.000000,0.001139,0.002572,0.000493,0.000000,0.000000
Uzbekistan,0.000000,0.000759,0.000000,0.000000,0.000000,0.000000


In [12]:
pd.crosstab(index=jb2.country_live, columns=jb2.age,
            normalize='index')

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,0.111111,0.388889,0.277778,0.166667,0.000000,0.055556
Argentina,0.009174,0.348624,0.403670,0.183486,0.045872,0.009174
Armenia,0.058824,0.764706,0.176471,0.000000,0.000000,0.000000
Australia,0.014599,0.211679,0.401460,0.229927,0.109489,0.032847
Austria,0.007812,0.242188,0.484375,0.171875,0.093750,0.000000
...,...,...,...,...,...,...
United States,0.014831,0.279199,0.386355,0.177234,0.097887,0.044494
Uruguay,0.000000,0.300000,0.650000,0.050000,0.000000,0.000000
Uzbekistan,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000


## 29.4 Hierarchical Columns with Cross Tabulations

- Let's look at the breakdown of country and age by where people use Python and Python version and then focus on the United States

In [13]:
(pd.crosstab(index=[jb2.country_live, jb2.age],
             columns=[jb2.use_python_most, jb2.python3_version_most])
    .loc[['United States']]
)

use_python_most        Computer graphics                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          0          0          0   
              21                       0          0          1          1   
              30                       0          0          0          1   
              40                       0          0          0          3   
              50                       0          0          1          2   
              60                       0          0          1          0   

use_python_most                       Data analysis                        \
python3_version_most Python 3_9 Python 3_5 or lower Python 3_6 Python 3_7   
country_live  age                                                           
United States 18              0                   0          0          1   
              21              1                   1         18         48   
              30              0                   3         29         66   
              40              0                   0         14         30   
              50              0                   2          6         26   
              60              0                   0          1         11   

use_python_most                             ...             Unknown  \
python3_version_most Python 3_8 Python 3_9  ... Python 3_5 or lower   
country_live  age                           ...                       
United States 18              5          0  ...                   0   
              21             64         11  ...                   0   
              30             90         12  ...                   0   
              40             45          3  ...                   2   
              50             36          1  ...                   1   
              60             11          1  ...                   0   

use_python_most                                                   \
python3_version_most Python 3_6 Python 3_7 Python 3_8 Python 3_9   
country_live  age                                                  
United States 18              1          0          1          0   
              21             10         13         11          4   
              30              7         19         14          4   
              40              5          5          2          4   
              50              2          3          6          0   
              60              4          2          3          0   

use_python_most          Web development                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          0          1          4   
              21                       3         28         54         81   
              30                       3         60         77        129   
              40                       1         14         26         66   
              50                       0         10         12         14   
              60                       0          3          5          5   

use_python_most                  
python3_version_most Python 3_9  
country_live  age                
United States 18              0  
              21              4  
              30             14  
              40              8  
              50              2  
              60              1  

[6 rows x 84 columns]

In [15]:
# just look at data analysis and web development
(pd.crosstab(index=[jb2.country_live, jb2.age],
             columns=[jb2.use_python_most, jb2.python3_version_most])
    .loc[['United States'],  ['Data analysis', 'Web development']]
)

use_python_most            Data analysis                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          0          1          5   
              21                       1         18         48         64   
              30                       3         29         66         90   
              40                       0         14         30         45   
              50                       2          6         26         36   
              60                       0          1         11         11   

use_python_most                     Web development                        \
python3_version_most Python 3_9 Python 3_5 or lower Python 3_6 Python 3_7   
country_live  age                                                           
United States 18              0                   0          0          1   
              21             11                   3         28         54   
              30             12                   3         60         77   
              40              3                   1         14         26   
              50              1                   0         10         12   
              60              1                   0          3          5   

use_python_most                             
python3_version_most Python 3_8 Python 3_9  
country_live  age                           
United States 18              4          0  
              21             81          4  
              30            129         14  
              40             66          8  
              50             14          2  
              60              5          1

## 29.5 Heatmaps

In [16]:
(pd.crosstab(index=[jb2.country_live, jb2.age],
             columns=[jb2.use_python_most, jb2.python3_version_most])
    .loc[['United States'],  ['Data analysis', 'Web development']]
    .style.background_gradient(cmap='viridis', axis=None)
)